# Basic Backtest Example

This notebook demonstrates the simplest way to use `rademacher-backtest`:
1. Load price data from a CSV file
2. Run a backtest with a simple portfolio
3. Analyze the results

## Installation

```bash
pip install rademacher-backtest
```

## 1. Load Price Data

First, let's load the sample price data using pandas:

In [ ]:
import pandas as pd
import rademacher_backtest as rbt

# Load price data
prices = pd.read_csv('data/sample_prices.csv', index_col='date', parse_dates=True)

print(f"Loaded {len(prices)} days of price data")
print(f"Tickers: {', '.join(prices.columns)}")
print(f"\nDate range: {prices.index[0].date()} to {prices.index[-1].date()}")

# Show first few rows
prices.head()

## 2. Run a Simple Backtest

Now let's run a backtest with a classic 60/40 portfolio (60% stocks, 40% bonds):

In [ ]:
# Create a data loader from the DataFrame
loader = rbt.DataFrameLoader(prices)

# Define a 60/40 portfolio (60% SPY stocks, 40% AGG bonds)
portfolio = {
    'SPY': 0.60,  # S&P 500 ETF
    'AGG': 0.40,  # Aggregate Bond ETF
}

# Run the backtest
result = rbt.backtest(
    portfolio=portfolio,
    loader=loader,
    start_date='2015-01-01',
    end_date='2023-12-31',
    initial_capital=100_000.0,
    transaction_cost_bps=10.0,  # 0.1% transaction cost
)

print(f"\n✅ Backtest complete!")
print(f"Initial value: ${result.initial_value:,.2f}")
print(f"Final value: ${result.final_value:,.2f}")
print(f"Total return: {result.total_return_pct:.2f}%")

## 3. Calculate Performance Metrics

Let's calculate standard performance metrics:

In [ ]:
# Calculate performance metrics
perf_calc = rbt.PerformanceCalculator()
metrics = perf_calc.calculate(result.daily_returns)

# Display key metrics
print("\n📊 Performance Metrics:")
print(f"  CAGR: {metrics.cagr:.2f}%")
print(f"  Volatility (Annual): {metrics.annualized_volatility:.2f}%")
print(f"  Sharpe Ratio: {metrics.sharpe_ratio:.3f}")
print(f"  Max Drawdown: {metrics.max_drawdown:.2f}%")
print(f"  Calmar Ratio: {metrics.calmar_ratio:.3f}")

## 4. RAS Analysis

Now let's apply the Rademacher Anti-Serum (RAS) methodology to get statistically rigorous bounds on performance:

In [ ]:
# Perform RAS analysis
ras_result = rbt.analyze_ras(
    returns=result.daily_returns,
    confidence=0.99,  # 99% confidence level
    n_strategies=1,   # Single strategy (no multiple testing)
)

print("\n🔬 RAS Analysis:")
print(f"  Empirical Sharpe: {ras_result.empirical_sharpe_annualized:.3f}")
print(f"  RAS-Adjusted Sharpe: {ras_result.adjusted_sharpe_annualized:.3f}")
print(f"  Total Haircut: {ras_result.total_haircut_annualized:.3f}")
print(f"  Statistically Positive: {'Yes ✅' if ras_result.is_statistically_positive else 'No ❌'}")

print(f"\n{ras_result.interpretation}")

## 5. View Daily Returns

Let's plot the cumulative returns over time:

In [ ]:
# Simple plot using pandas (no additional dependencies)
cumulative_returns = (1 + result.daily_returns).cumprod()

# Display plot
cumulative_returns.plot(figsize=(12, 6), title='Cumulative Returns - 60/40 Portfolio')
import matplotlib.pyplot as plt
plt.ylabel('Cumulative Return')
plt.xlabel('Date')
plt.grid(True, alpha=0.3)
plt.show()

## Summary

This example showed:
1. ✅ Loading price data from CSV
2. ✅ Running a backtest with a simple portfolio
3. ✅ Calculating performance metrics
4. ✅ Applying RAS methodology for statistical rigor
5. ✅ Visualizing results

The RAS-adjusted Sharpe ratio provides a conservative estimate that accounts for:
- Estimation error from finite sample size
- Complexity penalty from model selection
- Multiple testing correction (if applicable)

## Next Steps

- See `02_custom_portfolio.ipynb` for portfolio comparison
- See `03_ras_analysis.ipynb` for deep dive into RAS methodology
- See `04_visualization.ipynb` for advanced charts (requires `[viz]` extra)